# Examples of operations on the classes in the "io" package.

This notebook should eventually turn into documentation.

In [ ]:
%pip install transformers[torch]

In [ ]:
import json

import aconfig

In [ ]:
INPUT_JSON_STRS = [
    """
{
    "messages":
    [
        {"role": "user", "content": "Hello, how are you?"},
        {"role": "assistant", "content": "I'm doing great. How can I help you today?"},
        {"role": "user", "content": "I'd like to show off how chat templating works!"}
    ]
}
""",
    """
{
    "messages":
    [
        {"role": "user", "content": "How much wood could a wood chuck chuck?"}
    ],
    "thinking": true
}
""",
]

input_json_str = INPUT_JSON_STRS[1]
print(input_json_str)

In [ ]:
input_json = json.loads(input_json_str)
input_json

In [ ]:
from granite_io.io.base import ChatCompletionInputs

input_obj = ChatCompletionInputs.model_validate(input_json)
input_obj

In [ ]:
from granite_io.io.granite_3_2.input_processors.granite_3_2_input_processor import (
    Granite3Point2Inputs,
)  # noqa: E501

granite_input_obj = Granite3Point2Inputs.model_validate(input_obj.model_dump())
granite_input_obj

In [ ]:
reconstituted_json = input_obj.model_dump_json(indent=4)
print(reconstituted_json)

In [ ]:
import transformers

GRANITE_3_2_MODEL_STR = "ibm-granite/granite-3.2-2b-instruct"

tokenizer = transformers.AutoTokenizer.from_pretrained(GRANITE_3_2_MODEL_STR)

input_kwargs = input_json.copy()
del input_kwargs["messages"]
transformers_str = tokenizer.apply_chat_template(
    input_json["messages"], **input_kwargs, tokenize=False, add_generation_prompt=True
)

print(transformers_str)

In [ ]:
from granite_io.io.granite_3_2.granite_3_2 import Granite3Point2InputOutputProcessor

inputs = ChatCompletionInputs.model_validate_json(input_json_str)
io_proc_str = Granite3Point2InputOutputProcessor().inputs_to_string(inputs)
print(io_proc_str)

In [ ]:
# Load a model onto the GPU and wrap it in an I/O processor.

import torch

from granite_io.io.granite_3_2.granite_3_2 import Granite3Point2InputOutputProcessor
from granite_io.backend.transformers import TransformersBackend

if torch.cuda.is_available():
    device_name = "cuda"
# Re-enable once we have a smaller model that can run in a laptop GPU
elif torch.backends.mps.is_available():
    device_name = "mps"
else:
    device_name = "cpu"
    # CPU mode; prevent thrashing
    torch.set_num_threads(4)

backend = TransformersBackend(
    aconfig.Config(
        {"model_name": GRANITE_3_2_MODEL_STR, "device": device_name},
        override_env_vars=False,
    ),
)
io_processor = Granite3Point2InputOutputProcessor(backend=backend)

In [ ]:
result = io_processor.create_chat_completion(inputs)
print(result.results[0].next_message.content)